# Reuben Rapose
# J040

# **Questions :**


> ### 1. Denormalize data --> Not done
> ### 2. Topic Modelling(LDA & NMF) --> **DONE**
> ### 3. Transformer Function & Pipeline --> **DONE**
> ### 4. Visualization using pyLDAvis --> **DONE**
> ### 5. Extracting columns and calculating cosine similarity --> **DONE...kinda** 



I have not Copied or given any of my friends a copy of this notebook...

# Have Imported all functions used in the notebook together in one cell

In [0]:
import pandas as pd
import numpy as np
import nltk
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
import re
from gensim import corpora
from gensim import corpora, models
from pprint import pprint
import pickle
import nltk
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import NMF
from nltk.stem import WordNetLemmatizer 
from sklearn.feature_extraction.text import TfidfTransformer,CountVectorizer,TfidfVectorizer
from nltk import word_tokenize
import string 
import pyLDAvis.gensim
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import nmf
from nltk.corpus import stopwords
from nltk.tokenize import MWETokenizer,word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from nltk.corpus import wordnet
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split as tts,GridSearchCV
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from gensim.models import word2vec


In [0]:
# %reset #ignore

#1.0 - Reading the Dataset

In [0]:
#reading through my github
url = "https://raw.githubusercontent.com/reubence/IPYNB-Notebooks/master/datasets/Articles.csv"
df_art = pd.read_csv(url, encoding = "utf-8’")

In [0]:
#downloading corpus and other dependencies

nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('german'))
nltk.download('words')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [0]:
df_art.head()

,ID_Article,Path,publishingDate,Title,Body
0,1,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro..."
1,2,Newsroom/User/Community/Regeln,2012-05-26 12:12:19.46,Werden Sie Teil von derStandard.at!,"<div class=""diashow"" id=""objectContent""><meta ..."
2,3,Diverses/mobil,2013-11-22 12:15:00.00,Die Android App von derStandard.at,"<div class=""section"" id=""content-main"" itempro..."
3,4,Newsroom/User/mitmachen/Mitreden,2014-08-13 05:30:00.00,Welche Erfahrungen haben Sie als Linkshänder g...,"<div class=""section"" id=""content-main"" itempro..."
4,5,Newsroom/User/mitmachen/Mitreden,2014-08-27 12:27:01.09,Wie haben Sie das Jahr 1989 erlebt?,"<div class=""section"" id=""content-main"" itempro..."



#2.0 - Defining Preprocessing Functions in steps (SkLearn pipeline)

In [0]:
def to_lower(str): # 1.  Lowering and removing digits and separating gmail words because we dont want to read nicknames and emails
  text= str.lower() #1
  text = text.translate(text.maketrans('', '', string.digits))
  text = text.translate(text.maketrans('', '', '''!()-[]{};:'"\,<>./?@#$%^&*_~'''))
  str_new = " ".join(re.findall("[a-zA-Z]+", text))
  return str_new

<input>:4: DeprecationWarning: invalid escape sequence \,
<input>:4: DeprecationWarning: invalid escape sequence \,
<input>:4: DeprecationWarning: invalid escape sequence \,
<input>:4: DeprecationWarning: invalid escape sequence \,
<input>:4: DeprecationWarning: invalid escape sequence \,
<input>:4: DeprecationWarning: invalid escape sequence \,
<input>:4: DeprecationWarning: invalid escape sequence \,
<input>:4: DeprecationWarning: invalid escape sequence \,
<input>:4: DeprecationWarning: invalid escape sequence \,
<ipython-input-56-3603692189bb>:4: DeprecationWarning: invalid escape sequence \,
  text = text.translate(text.maketrans('', '', '''!()-[]{};:'"\,<>./?@#$%^&*_~'''))


In [0]:
 def tokenize(str_new):# 2. Tokenizing again because the result returns a string.. just to be safe
  tokens = [] #2
  tokens = nltk.word_tokenize(str_new)
  return tokens

In [0]:
def remove_stop(tokens):# 3. Removing stopwords using german corpus... also removinbg punctuations
  removed = [] #3
  stopword = stopwords.words('german') 
  punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
  for w in tokens:
    if (w.lower() not in stopword and w.lower() not in punctuations):
      removed.append(w.lower())
  return removed

<input>:4: DeprecationWarning: invalid escape sequence \,
<input>:4: DeprecationWarning: invalid escape sequence \,
<input>:4: DeprecationWarning: invalid escape sequence \,
<input>:4: DeprecationWarning: invalid escape sequence \,
<input>:4: DeprecationWarning: invalid escape sequence \,
<input>:4: DeprecationWarning: invalid escape sequence \,
<input>:4: DeprecationWarning: invalid escape sequence \,
<input>:4: DeprecationWarning: invalid escape sequence \,
<input>:4: DeprecationWarning: invalid escape sequence \,
<ipython-input-58-ad4368552d53>:4: DeprecationWarning: invalid escape sequence \,
  punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''


In [0]:
def pos_tag(removed):
           # 4. Pos Tagging for Lemmatizing
  pos = [] #4
  pos.append(nltk.pos_tag(removed))
  return pos

In [0]:
def lemma(pos):# 5. Lemmatizing
               
  lem = WordNetLemmatizer() #5
  lemma = []
  for word in pos:
    for w in word:
      pos_value = ""
      if (w[1].startswith('J')):
        pos_value = wordnet.ADJ
      elif (w[1].startswith('V')):
        pos_value = wordnet.VERB
      elif (w[1].startswith('N')):
        pos_value = wordnet.NOUN
      elif (w[1].startswith('R')):
        pos_value = wordnet.ADV 
      else:
        continue
      lemma.append(lem.lemmatize(w[0],pos_value))
  return lemma
 

In [0]:
def to_string(lemma): # 6. Return Sentence instead of tokens
  str1 = ' '.join([elem for elem in lemma]) #6
  return (str1)

#2.1 - Fitting Sklearn pipeline to ["Title"] column using Function Transformer

In [0]:
#defining steps for the pipeline

pipeline = make_pipeline(
        #Functions defined in above steps go here
        FunctionTransformer(to_lower),
        FunctionTransformer(tokenize),
        FunctionTransformer(remove_stop),
        FunctionTransformer(pos_tag),
        FunctionTransformer(lemma),
        FunctionTransformer(to_string),
    )


In [0]:
#Selecting first 500 rows only to save computation :]
df_art = df_art.head(500)
df_art.shape

(500, 5)

In [0]:
#Fitting the pipeline to our column ["Title"] and ["Body"]

df_art["Title-Cleaned"] = df_art["Title"].apply(pipeline.fit_transform) #storing it in new column ["Title-Cleaned"]
df_art["Title-Cleaned"] 

0                             newsletter derstandardat
1                                   teil derstandardat
2                            android app derstandardat
3                      erfahrungen linksh nder gemacht
4                                          jahr erlebt
                            ...                       
495            fiat brautschau h lt autobranche touren
496                  mann s penistransplantation vater
497           koller zuversichtlich harte russlandnuss
498     fox sky sterreich gewinnen walk dead promaxbda
499    dominique strausskahn bersteht letzte sexaff re
Name: Title-Cleaned, Length: 500, dtype: object

In [0]:
#Extra Function to remove <html> tags
def striphtml(data):
    p = re.compile(r'<.*?>')
    return p.sub('', data)

In [0]:
#Applying stip_html function

jaja  = df_art['Body'].apply(striphtml)

In [0]:
#Applying sklearn pipeline to preprocess column and returning it to dataframe

df_art["Body-Cleaned"]=  jaja.apply(pipeline.fit_transform)
df_art["Body-Cleaned"]

0      abonnieren unserer newsletter lassen aktuelle ...
1      teil derstandardat dezember function popupwinp...
2      smartphoneapp bietet nachrichten echtzeit m gl...
3      sieht alltag linksh nder erledigen arbeiten re...
4      erz hlen erlebnissen rund weltpolitischen umbr...
                             ...                        
495    seit wochen wirbt fiatchryslerchef marchionne ...
496    eingriff j hrigem weltpremierefreude nachwuchs...
497    sterreichs fu ball moskau f r em buchen reise ...
498    gemeinschaftsprojekt walk dead scary shelter s...
499    ehemalige iwfchef zuh ltereiprozess freispruch...
Name: Body-Cleaned, Length: 500, dtype: object

In [0]:
df_art.head()

,ID_Article,Path,publishingDate,Title,Body,Title-Cleaned,Body-Cleaned
0,1,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",newsletter derstandardat,abonnieren unserer newsletter lassen aktuelle ...
1,2,Newsroom/User/Community/Regeln,2012-05-26 12:12:19.46,Werden Sie Teil von derStandard.at!,"<div class=""diashow"" id=""objectContent""><meta ...",teil derstandardat,teil derstandardat dezember function popupwinp...
2,3,Diverses/mobil,2013-11-22 12:15:00.00,Die Android App von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",android app derstandardat,smartphoneapp bietet nachrichten echtzeit m gl...
3,4,Newsroom/User/mitmachen/Mitreden,2014-08-13 05:30:00.00,Welche Erfahrungen haben Sie als Linkshänder g...,"<div class=""section"" id=""content-main"" itempro...",erfahrungen linksh nder gemacht,sieht alltag linksh nder erledigen arbeiten re...
4,5,Newsroom/User/mitmachen/Mitreden,2014-08-27 12:27:01.09,Wie haben Sie das Jahr 1989 erlebt?,"<div class=""section"" id=""content-main"" itempro...",jahr erlebt,erz hlen erlebnissen rund weltpolitischen umbr...


#3.0 - CountVectorizer --> NMF --> LDA --> LDApyvis

#3.1 - CountVectorizer

In [0]:
#Vectorizing the processed Title column

vectorizer = CountVectorizer(input="str")
X = vectorizer.fit_transform(df_art['Title-Cleaned'])
feature_names = vectorizer.get_feature_names()
dense = X.todense()
denselist = dense.tolist()
df3 = pd.DataFrame(denselist, columns=feature_names)
df3.head()


,ab,abbauarbeiten,abgeworben,abgrund,abh,ablinger,absagen,abschlie,absolute,abstieg,abwehrschlacht,abwicklung,abzuwenden,achtelfinale,achtung,adligen,adolf,adressen,again,agb,akp,aktien,aktion,al,albanerparteichef,alberschwende,algier,alias,alkohol,alkolenker,allein,alpiner,alte,alter,altera,alternative,am,amnestyvorw,amoklaufen,amore,...,wochenlang,wohnen,wollten,world,wowspieler,wuchs,wwdc,xbox,yamaha,yogaclip,youtubewerberwahl,zahlt,zahnl,zarte,zehn,zehnte,zeichen,zeigen,zeigt,zeit,zeiten,zeitungen,zentralmatura,zerbricht,zerbrochen,ziehen,zieht,zinsflaute,zivilklage,zu,zukunft,zulieferern,zuordnen,zusammenhang,zuschauerzahl,zuversichtlich,zwei,zwergplanet,zwischenf,zwischengelandet
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
#Vectorizing the processed Body column

vectorizer_b = CountVectorizer(input="str")
X_b = vectorizer_b.fit_transform(df_art['Body-Cleaned'])
feature_names_b = vectorizer_b.get_feature_names()
dense_b = X_b.todense()
denselist_b = dense_b.tolist()
df3_b = pd.DataFrame(denselist_b, columns=feature_names_b)
df3_b.head()


,aaaaaaatemlos,aaagames,aab,aac,aan,ab,abac,abbau,abbauarbeiten,abberuftweiter,abbey,abbiegen,abbilden,abbildung,abblitzen,abbr,abbremsblase,abbremsen,abbremst,abbringen,abc,abcserie,abcsg,abcsgexperte,abdecken,abdel,abder,abdrabbu,abdrehen,abdullahzentrum,abdullahzentrums,abe,abeitaliens,abejapan,abend,abendessen,abendl,abends,abendstunden,abendzettel,...,zweitst,zweitwichtigste,zweiw,zwerg,zwergplanet,zwergplaneten,zwick,zwickt,zwielichtige,zwietracht,zwillingsbruders,zwillingsstadt,zwingen,zwingend,zwingendes,zwingt,zwischenbilanz,zwischendurch,zwischenf,zwischenfall,zwischenkriegszeit,zwischenl,zwischenlanden,zwischenmenschliche,zwischennutzungsprojekt,zwischenruf,zwischenrufe,zwischenrufen,zwischenrufs,zwischenspiele,zwischenstopp,zwischenzeit,zwischenzeitlich,zwischenzeitlichem,zwischenzeitlichen,zwist,zwitschern,zyklus,zylinder,zylinderkopf
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#3.2 - Topic modelling using NMF (Only on Title Column)

In [0]:
model = NMF(n_components=15, init='nndsvd', random_state=0, alpha=0.1, l1_ratio=0.5)
W = model.fit_transform(df3)
W

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.00905821],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.00905821],
       [0.        , 0.        , 0.08128296, ..., 0.        , 0.        ,
        0.00904131],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.01623856, 0.        ,
        0.        ],
       [0.        , 0.02845779, 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [0]:
H = model.components_
H

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.14073945, 0.        ,
        0.        ],
       ...,
       [0.02256644, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [0]:
dfw=pd.DataFrame(W)

In [0]:
dfw.shape

(500, 15)

In [0]:
def display(H, W, feature_names, documents, no_top_words, no_top_documents):
    for topic_idx, topic in enumerate(H):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
        for i in topic.argsort()[:-no_top_words - 1:-1]]))
        top_doc_indices = np.argsort( W[:,topic_idx] )[::-1][0:no_top_documents]
        for doc_index in top_doc_indices:
            print(documents[doc_index])


In [0]:
#Printing the 15 topics and top keywords that define them

display(H,W,list(df3),df_art["Title-Cleaned"],1,5)

Topic 0:
apple
angriff aufs radio apple music gl cklich
mann erhielt strafzettel wegen apple watch auto
apple music maximale bitrate niedriger rivalen
apple co warum uskonzerne riesige geldreserven horten
apple music neue streamingdienst bringt
Topic 1:
fp
landtagswahl sp fp koalition burgenland abschlie en
aufregung geteiltes post steirischer fp bezirksgruppe
foto fp protest kurier klagt strache
koalition re handlanger fp
teilnehmer fp demo asylquartier erdberg
Topic 2:
neue
android tv neue kan le f r google smarttvsystem
neue ansicht mysteri sen flecken zwergplanet cere
android m erster blick neue softwaregeneration
neue k mpfe ukraine feuer fronten
neue k mpfe ostukraine tschechows gewehr
Topic 3:
ber
kleinstadth lle supermamas al bundy lasst ber familie serien reden
merkel obama netz macht ber foto ggipfel lustig
bloodstained bisher erfolgreichstes kickstartergame sammelt ber millionen dollar
deutscher bundestag schon l nger ber cyberangriffausma informiert
ber fl chtlinge wochenen

#3.3 - Topic modelling using LDA (sklearn) ... (Only on Title column) 
Have done topic modelling on body and title later on using gensim use

In [0]:
# LDA

LD=LatentDirichletAllocation(n_components=15,random_state=444) #Only 15 topics because I wanted to save time spent on running code 
fi=LD.fit_transform(X)
df_topic= pd.DataFrame(fi)
df_topic

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.022222,0.022222,0.022222,0.022222,0.022222,0.688889,0.022222,0.022222,0.022222,0.022222,0.022222,0.022222,0.022222,0.022222,0.022222
1,0.022222,0.022222,0.022222,0.022222,0.022222,0.022222,0.022222,0.022222,0.022222,0.022222,0.022222,0.688889,0.022222,0.022222,0.022222
2,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.766666,0.016667,0.016667,0.016667,0.016667
3,0.013333,0.813333,0.013333,0.013333,0.013333,0.013333,0.013333,0.013333,0.013333,0.013333,0.013333,0.013333,0.013333,0.013333,0.013333
4,0.022222,0.022222,0.022222,0.022222,0.022222,0.022222,0.022222,0.022222,0.688889,0.022222,0.022222,0.022222,0.022222,0.022222,0.022222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.844444,0.011111,0.011111,0.011111,0.011111
496,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.766667,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667
497,0.013333,0.013333,0.013333,0.013333,0.013333,0.013333,0.013333,0.013333,0.013333,0.013333,0.013333,0.013333,0.013333,0.013333,0.813333
498,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.883333


In [0]:
def dispplay(model, count_vectorizer, n_top_words):
    words = count_vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([words[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))

In [0]:
dispplay(LD,vectorizer,9)
#It's working!


Topic #0:
neue frau st ck rfe ukraine tote wochenende weist

Topic #1:
fp koalition hriger razzia king pr merkel austria blatters

Topic #2:
ck raketen vp haidermaurer wien fp sen burgenland mehr

Topic #3:
pr apple fp sp clinton seid erdogan co hund

Topic #4:
sterreich zwei fair bank hrer neues bleiben ggegner film

Topic #5:
derstandardat erh ger ht gr pleite erste geht protest

Topic #6:
st wiener ck seestadt community japan sch millionen fl

Topic #7:
apple gro sieht deutsche gewinnt facebook gefahr fu legt

Topic #8:
neuer tote sterreicher gr verliert google league premier belastet

Topic #9:
geht mehr russische lt chtig big sommer wirtschaft regierung

Topic #10:
nder sterreich achtelfinale lt steiermark wien eis ab system

Topic #11:
neue android sterreich usa linz gl offtopicforum rom fl

Topic #12:
sp ber gesch rebellen leben wien hrer geht gibt

Topic #13:
jahre ber juni putin chster wien macht schluckt merkel

Topic #14:
sp deutschland gewinnen wiener sky merkels city doll

#3.3 - pyLDAvis a.k.a Visualizing the models 
I have done visualization Using gensim instead of sklearn because i was getting some weird errors while using sklearn

In [0]:
!pip install pyLDAvis

In [0]:
#need to pass tokens... so i had to convert sentence to tokens again before creating corpus and dict
#Converting processed Title column to token

tokens = df_art["Title-Cleaned"].apply(nltk.word_tokenize) 

In [0]:
#need to define dictionary of words (bow) and corpus for doing Topic Modelling...pickling and also saving model for use

dictionary = corpora.Dictionary(tokens) #corpus
corpus = [dictionary.doc2bow(text) for text in tokens]
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
#need to pass tokens... so i had to convert sentence to tokens again before creating corpus and dict
#Converting processed Body column to token

tokens_b = df_art["Body-Cleaned"].apply(nltk.word_tokenize) 

In [0]:
#Creating same dictionary and all for Body column so that we can run Topic Modelling

dictionary_b = corpora.Dictionary(tokens_b) #corpus
corpus_b = [dictionary_b.doc2bow(text) for text in tokens_b]

pickle.dump(corpus_b, open('corpus.pkl', 'wb'))
dictionary_b.save('dictionary.gensim')


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
#LDA using gensim...initializing and saving model... 

NUM_TOPICS = 15
ldamodel = models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15,alpha = 0.1)
ldamodel.save('model5.gensim')
topics = ldamodel.print_topics(num_words=20)



Streaming output truncated to the last 5000 lines.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Us

In [0]:
topics

[(0,
  '0.057*"r" + 0.047*"f" + 0.007*"apple" + 0.007*"bringt" + 0.007*"hund" + 0.006*"jenner" + 0.003*"rom" + 0.003*"blatters" + 0.003*"festplattenabgabe" + 0.003*"haidermaurer" + 0.003*"polizei" + 0.003*"urheberrecht" + 0.003*"vier" + 0.003*"dollar" + 0.003*"king" + 0.003*"microsoft" + 0.003*"caitlyn" + 0.003*"endlich" + 0.003*"bl" + 0.003*"aufgabe"'),
 (1,
  '0.019*"r" + 0.016*"f" + 0.012*"wien" + 0.008*"neue" + 0.008*"leben" + 0.004*"sterreich" + 0.004*"hrer" + 0.004*"gr" + 0.004*"io" + 0.004*"stirbt" + 0.004*"koalition" + 0.004*"k" + 0.004*"jailbreak" + 0.004*"fehlen" + 0.004*"chtiger" + 0.004*"linzer" + 0.004*"ffnet" + 0.004*"kurz" + 0.004*"kommen" + 0.004*"feuer"'),
 (2,
  '0.016*"mehr" + 0.008*"pr" + 0.008*"sen" + 0.008*"league" + 0.005*"f" + 0.005*"r" + 0.004*"verliert" + 0.004*"w" + 0.004*"russische" + 0.004*"clinton" + 0.004*"t" + 0.004*"alternative" + 0.004*"upl" + 0.004*"premier" + 0.004*"ausl" + 0.004*"linz" + 0.004*"lt" + 0.004*"faymann" + 0.004*"ablinger" + 0.004*"unive

In [0]:
#Visualising using Py LDA vis... 
#I have only visualised the Title for demonstration because visualisation was taking a lot of time

lda = models.ldamodel.LdaModel.load('model5.gensim')

lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

# VOILA!

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [0]:
#Calculating probability score for each doc belonging to a particular class


tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

i = 0
asf = []
for doc in corpus_tfidf:
  pprint(doc)
  asf.append(doc)

[(0, 0.5798347381061715), (1, 0.8147341139815785)]
[(0, 0.5798347381061715), (2, 0.8147341139815785)]
[(0, 0.4899312130918076), (3, 0.5348457080104431), (4, 0.6884093804279527)]
[(5, 0.5340270796445955),
 (6, 0.5340270796445955),
 (7, 0.5340270796445955),
 (8, 0.38005951457474985)]
[(9, 0.7071067811865476), (10, 0.7071067811865476)]
[(11, 0.7071067811865476), (12, 0.7071067811865476)]
[(13, 0.6641867433319081), (14, 0.7475666993534116)]
[(15, 0.7071067811865476), (16, 0.7071067811865476)]
[(17, 0.4673336213606923),
 (18, 0.41520950075696916),
 (19, 0.4673336213606923),
 (20, 0.4673336213606923),
 (21, 0.41520950075696916)]
[(22, 0.40137483069491103),
 (23, 0.40137483069491103),
 (24, 0.2594655375005537),
 (25, 0.40137483069491103),
 (26, 0.3566074330026032),
 (27, 0.40137483069491103),
 (28, 0.40137483069491103)]
[(29, 0.6197424117602286), (30, 0.6197424117602286), (31, 0.4814962991874709)]
[(3, 0.4418233776084921),
 (32, 0.5052510347900875),
 (33, 0.36761777568883147),
 (34, 0.4681482

In [0]:
#Calculating probability score for each doc belonging to a particular class
# This one is doing it for Body column instead of text

tfidf = models.TfidfModel(corpus_b)
corpus_tfidf = tfidf[corpus_b]
i = 0
bsf = []
for doc in corpus_tfidf:
  pprint(doc)
  bsf.append(doc)

Streaming output truncated to the last 5000 lines.
 (8289, 0.06838947460269945),
 (9036, 0.0487878123937413),
 (9052, 0.053703474064772355),
 (9115, 0.05527797103959265),
 (9352, 0.06838947460269945),
 (10366, 0.0570629884876419),
 (10484, 0.1367789492053989),
 (10835, 0.053703474064772355),
 (11337, 0.10204190721037534),
 (11874, 0.047797153787297264),
 (11950, 0.06156087322731412),
 (13253, 0.0570629884876419),
 (14554, 0.0738096404402819),
 (14560, 0.059123639902354815),
 (14585, 0.06838947460269945),
 (15066, 0.06454380573993729),
 (15737, 0.06454380573993729),
 (16031, 0.06454380573993729),
 (18745, 0.06156087322731412),
 (19892, 0.06454380573993729),
 (20854, 0.06454380573993729),
 (22950, 0.0738096404402819),
 (24289, 0.06838947460269945),
 (24335, 0.0570629884876419),
 (25403, 0.06838947460269945),
 (25679, 0.0738096404402819),
 (25751, 0.06838947460269945),
 (28697, 0.1476192808805638),
 (31761, 0.08307547514062653),
 (31762, 0.08307547514062653),
 (31763, 0.08307547514062653)

In [0]:
#Merging everything into our original DataFrame

df_art["Topic-Modelling-Scores-Body"] = bsf
df_art["Topic-Modelling-Scores-Title"] = asf
# df_art.shape
df_art.head()

,ID_Article,Path,publishingDate,Title,Body,Title-Cleaned,Body-Cleaned,Topic-Modelling-Scores-Body,Topic-Modelling-Scores-Title
0,1,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",newsletter derstandardat,abonnieren unserer newsletter lassen aktuelle ...,"[(0, 0.07202932793523531), (1, 0.0720293279352...","[(0, 0.5798347381061715), (1, 0.81473411398157..."
1,2,Newsroom/User/Community/Regeln,2012-05-26 12:12:19.46,Werden Sie Teil von derStandard.at!,"<div class=""diashow"" id=""objectContent""><meta ...",teil derstandardat,teil derstandardat dezember function popupwinp...,"[(1, 0.042555637317873714), (25, 0.41199299778...","[(0, 0.5798347381061715), (2, 0.81473411398157..."
2,3,Diverses/mobil,2013-11-22 12:15:00.00,Die Android App von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",android app derstandardat,smartphoneapp bietet nachrichten echtzeit m gl...,"[(2, 0.06939694397186333), (3, 0.0964196307200...","[(0, 0.4899312130918076), (3, 0.53484570801044..."
3,4,Newsroom/User/mitmachen/Mitreden,2014-08-13 05:30:00.00,Welche Erfahrungen haben Sie als Linkshänder g...,"<div class=""section"" id=""content-main"" itempro...",erfahrungen linksh nder gemacht,sieht alltag linksh nder erledigen arbeiten re...,"[(6, 0.04548378611271271), (13, 0.006531766884...","[(5, 0.5340270796445955), (6, 0.53402707964459..."
4,5,Newsroom/User/mitmachen/Mitreden,2014-08-27 12:27:01.09,Wie haben Sie das Jahr 1989 erlebt?,"<div class=""section"" id=""content-main"" itempro...",jahr erlebt,erz hlen erlebnissen rund weltpolitischen umbr...,"[(13, 0.008675734794946112), (25, 0.0987602139...","[(9, 0.7071067811865476), (10, 0.7071067811865..."




> **So basically Each row contains an array-wihtin-an-array with one value of probability for each class**



In [0]:
len(asf[55])

4

Tried removing cosine similarity but was running into errors... 
Besides, You mentioned in classs that even if we give you the columns it is fine... :)

In [0]:
from sklearn.metrics.pairwise import cosine_similarity
# (df_art.groupby(['Topic-Modelling-Scores-Title','Topic-Modelling-Scores-Body']).apply(lambda g: cosine_similarity(g['APerc'], g['VPerc'])[0][0]))
cosine_similarity(asf, bsf)

for row in range(100):
  for tow in range(100) :
    cosine_similarity(asf[[row,tow]],bsf[[row,tow]])

ValueError: ignored

So then I tried removing similarity using inbuilt functions of gensim... hence the cells below

In [0]:
#Coherance Score for Title Topic Modelling...irrelavant i know.. but better than nothing

from gensim.models import CoherenceModel

coherence_model_lda = CoherenceModel(model=lda, texts=tokens, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)



Coherence Score:  0.6549711431596827


In [0]:
#same as above but for Body column

from gensim.models import CoherenceModel

coherence_model_lda_b = CoherenceModel(model=lda, texts=tokens_b, dictionary=dictionary_b, coherence='c_v')
coherence_lda_b = coherence_model_lda_b.get_coherence()
print('\nCoherence Score: ', coherence_lda_b)



Coherence Score:  0.5589745336267561


In [0]:
#words categorized by topics 

for idx, topic in LD.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

AttributeError: ignored